## MultiLayer Perceptron

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import torch
import numpy as np
import torch.nn as nn
import math
from sklearn.metrics import mean_squared_error
import numpy as np
import seaborn as sns
import netCDF4 as nc
import xarray as xr
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import shiftgrid
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class MLPNetwork():

    def __init__(self, input_dim):
        super(nn.Module, self).__init__()

        self.input_dim = input_dim

        self.layers = nn.Sequential(
            nn.Linear(self.input_dim, 32),
            F.relu(),
            nn.Linear(32, 32),
            F.relu(),
            nn.Linear(32, 16),
            F.relu(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
model = MLPNetwork()

In [ ]:
# Loss Function: Mean Squared Error
lossfunc = nn.MSELoss()

# SGD = Stochastic Gradient Descent
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def training_and_evaluate(model, epochs, loss_function, optimizer, X_train, y_train, X_test, verbose=True):
    # A variavel history_train e utilizada para armazenar a perda em cada epoca
    history_train = np.zeros(epochs)

    # colocar no modelo no tipo treino
    model.train()

    # o modelo sera treinado utilizando um batch, portanto, ha apenas um 
    # laço para iterar as epocas no procedimento de teste 
    for epoch in range(epochs):

        # forward pass
        y_pred = model.forward(X_train)

        # funcao perda
        loss = loss_function(y_pred, y_train)

        # o modo verboso imprime a informacao de treinamento
        if verbose:
            print(f"Epoch [{epoch + 1}|{epochs}] Loss: {loss.item()}")

        # inclui a perda da epoca atual no history_train
        history_train[epoch] = loss.item()

        # limpa o gradiente
        optimizer.zero_grad()

        # atualiza os parametros
        loss.backward()
        optimizer.step()

    # coloca o modelo em modo de validacao
    model.eval()

    # desativa o caluclo do gradiente. isso e util para procedimentos inferenciais.
    # doc.: (https://pytorch.org/docs/stable/generated/torch.no_grad.html)
    with torch.no_grad():
        # realiza predicoes para os conjuntos de treino e de teste
        y_pred_train = model.forward(X_train)
        y_pred_test = model.forward(X_test)

    return history_train, y_pred_train, y_pred_test